# Initialization

In [1]:
%reload_ext autoreload
%autoreload 2

In [6]:
from result_saver import SaverProvider

provider = SaverProvider()

# Initialize simulator

In [18]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
DISTANCE = 7
ROUNDS = 7
OTHER_DATE = '2023-10-27'
_RESETS = False
LOGICAL = 0 # NOT NEEDED FOR EXTREME IQ BCS HARDCODED 0

NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]
# NOISE_LIST = [noise * 2 for noise in NOISE_LIST]

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, _resets = _RESETS, other_date=OTHER_DATE)

# Loop function

In [27]:
from tqdm import tqdm
import json
import cpp_soft_info

def func(distance, resets, shots = 1e5, interval_offset = 0.5, nbs_intervals = None):
    # Initialize simulator
    simulator = RepCodeIQSimulator(provider, distance, distance, DEVICE, _is_hex=_is_hex, _resets = resets, other_date=OTHER_DATE)
    IQ_data= simulator.generate_IQ(int(shots), noise_list=NOISE_LIST)

    # Run decoding
    num_errors = []
    bits = list(range(20))
    nb_intervals = [2**bit for bit in bits[1:]]
    if nbs_intervals is not None:
        nb_intervals = [nb for nb in nbs_intervals]
    nb_intervals.append(-1)
    for nb_intervals in tqdm(nb_intervals):
        result = cpp_soft_info.decode_IQ_fast(simulator.stim_circ.detector_error_model(decompose_errors=False), IQ_data,
                                            distance, int(LOGICAL), resets, simulator.qubit_mapping, simulator.grid_dict,
                                            simulator.processed_scaler_dict, _detailed=False, nb_intervals=nb_intervals, interval_offset=interval_offset)
        num_errors.append(result.num_errors)

    # Save results
    metadata = {"DEVICE": DEVICE, "DISTANCE": distance, "ROUNDS": distance, "OTHER_DATE": OTHER_DATE, 
                "_RESETS": resets, "LOGICAL": LOGICAL, "SHOTS": int(shots), "interval_offset": interval_offset}    
    with open(f"result_tradeoff_grid_intervals.json", "r") as f:
        data = json.load(f)
        data_in_str = [str(d) for d in data]
        new_data = {"metadata": metadata, "num_errors": num_errors, "bits": bits[1:]+[-1], "nbs_intervals": nbs_intervals}
        if str(new_data) not in data_in_str:
            data.append(new_data)
        else:
            print("Already in data! Not appending to file")

    with open(f"result_tradeoff_grid_intervals.json", "w") as f:
        json.dump(data, f, indent=1)


In [31]:
import numpy as np
distances = np.arange(3, 56, 4)
distances = np.arange(5, 13, 2)
distances = [7, 10, 13]

nbs_intervals = np.linspace(2, 21, 20).astype(int)

print(distances)
print(nbs_intervals)

[7, 10, 13]
[ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]


In [32]:
import numpy as np

# distances = [3, 7, 15, 23, 31]


for distance in distances[::-1]:
    print(f"\nDistance: {distance}")
    func(int(distance), resets=True, shots=1e6, interval_offset=0.5, nbs_intervals=nbs_intervals)
    # func(int(distance), resets=True, shots=2e5, interval_offset=0.5)


Distance: 13


100%|██████████| 21/21 [20:14<00:00, 57.81s/it]



Distance: 10


100%|██████████| 21/21 [11:59<00:00, 34.28s/it]



Distance: 7


100%|██████████| 21/21 [05:54<00:00, 16.88s/it]
